## Maps address to geolocation data

### Takes about 10 minutes to execute

In [1]:
#system dependent - some package was problematic
import sys
print(sys.path)
sys.path.append('/home/upflo/.local/lib/python3.6/site-packages')

import pandas as pd

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut


['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/upflo/fakultet/psz/venv/lib/python3.6/site-packages', '/home/upflo/fakultet/psz/venv/lib/python3.6/site-packages/IPython/extensions', '/home/upflo/.ipython']


In [2]:
df = pd.read_csv('../podaci/data_odbrana', lineterminator='\n', header='infer')
df = df[(df.tip == 'stan') & (df.grad=='Beograd') & (df.usluga =='prodaja')]

In [3]:
geolocator = Nominatim(user_agent="psz_projekat")
def geocode(adresa, recursion=0):
    try:
        return geolocator.geocode(adresa)
    except BaseException as e:
        if recursion > 10:      # max recursions
            raise e

        time.sleep(1) # wait a bit
        # try again
        return geocode(adresa, recursion=recursion + 1)

In [4]:
adresa_to_geoloc = dict()
uniq = df.adresa.unique()
for adresa in uniq:
    adresa_to_geoloc[adresa] = geocode('Beograd' + ' ' + adresa)
    
df['latitude'] = df['adresa'].map(lambda adresa:adresa_to_geoloc[adresa].latitude if adresa in uniq and adresa_to_geoloc[adresa] else None)
df['longitude'] = df['adresa'].map(lambda adresa: adresa_to_geoloc[adresa].longitude if adresa in uniq and adresa_to_geoloc[adresa] else None)

df = df[df.latitude.notna()]

knez_mihailova = geocode('Knez Mihailova')
knez_mihailova_lat_long = (knez_mihailova.latitude, knez_mihailova.longitude)

from geopy import distance

def calculate_distance(row):
    centar = (44.8178925, 20.4569641)
    stan = (row.latitude, row.longitude)
    row.udaljenost_od_centra = distance.distance(centar, stan).km
    return row

df['udaljenost_od_centra'] = 0.0
df = df.apply(calculate_distance, axis=1)

In [5]:
df.head()

,usluga,tip,cena,grad,sira_lokacija,uza_lokacija,adresa,povrsina,plac,broj_soba,...,orijentacija_nekretnine,tip_objekta,tip_stana,tip_kuce,sifra_oglasa,opis,link,latitude,longitude,udaljenost_od_centra
3,prodaja,stan,2.000 €,Beograd,Palilula opština,Krnjača,Partizanski blok ulica 1.,50m²,NaN,2 sobe,...,NaN,NaN,NaN,NaN,1080,U prilici smo da Vam ponudimo stanove građene ...,https://www.4zida.rs/prodaja/stanovi/beograd/o...,44.842779,20.483710,3.481600
5,prodaja,stan,275.000 €,Beograd,Zvezdara opština,Lion,Batutova,162m²,NaN,6 soba,...,NaN,NaN,NaN,NaN,1958,AGENCIJSKA PROVIZIJA 2% OPIS NEKRETNINE : uknj...,https://www.4zida.rs/prodaja/stanovi/beograd/o...,44.797302,20.499415,4.063796
6,prodaja,stan,270.000 €,Beograd,Palilula opština,Palilula,Dr Dragoslava Popovića,105m²,NaN,4 sobe,...,NaN,NaN,NaN,NaN,102447624,"Agencijski ID: 47624, III/V, klima, lift, tele...",https://www.4zida.rs/prodaja/stanovi/beograd/o...,44.811951,20.474358,1.526104
8,prodaja,stan,610.000 €,Beograd,Vračar,Neimar,Braničevska,153m²,NaN,4 sobe,...,NaN,NaN,NaN,NaN,65,Izuzetan stan u novogradnji vrhunskog kvalitet...,https://www.4zida.rs/prodaja/stanovi/beograd/o...,44.794506,20.469430,2.779669
10,prodaja,stan,193.000 €,Beograd,Stari Grad opština,Dorćol,Dobračina,72m²,NaN,2.5 sobe,...,NaN,NaN,NaN,NaN,NaN,U centru grada na odličnoj lokaciji prodajemo ...,https://www.4zida.rs/prodaja/stanovi/beograd/o...,44.819149,20.462108,0.430149


In [6]:
df.shape

(3021, 34)

In [7]:
df.to_csv('../podaci/adr_to_distance_odbrana.csv', encoding='utf-8', index=False)